In [1]:
import os
import cv2
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import torch.optim as optim

rdir_path = '../input/d123456/dataset'
imgs = []
key = []
l = -1
get_dir = [i for i in os.listdir(rdir_path) if os.path.isdir(os.path.join(rdir_path, i))]
for d in get_dir:
    l = l + 1
    path = rdir_path+'/'+d
    get_files = [i for i in os.listdir(path) if os.path.isfile(os.path.join(path, i))]
    
    for file in get_files:
        img = path+'/'+file
        imgs.append((img))       
        key.append(l)

data = (imgs,key)
train_x,test_x, train_y, test_y = train_test_split(data[0], data[1], train_size=0.7, random_state=42,shuffle = True,stratify = data[1])
train_y=np.array(train_y)
test_y=np.array(test_y)

In [2]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.6.0" to /root/.cache/torch/hub/v0.6.0.zip
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [3]:
def getdata(img):
    transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    for i in range(len(img)):
        temp = Image.open(img[i])
        temp=transform(temp)
        img[i]=temp
    torch_set=torch.stack(img)
    return torch_set

train_data = getdata(train_x)
test_data = getdata(test_x)

In [4]:
def model_data(images):
    features = []
    for image in images:
        image = image.unsqueeze(0)
        with torch.no_grad():
            feature = AlexNet_model(image)
            feature = np.squeeze(feature.numpy())
            features.append(feature)
    return features
x_train=np.array(model_data(train_data))
x_test=np.array(model_data(test_data))


In [5]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(x_train,train_y)

SVC(kernel='linear')

In [6]:
from sklearn.metrics import accuracy_score

y_pred = svc.predict(x_test)
print(accuracy_score(test_y,y_pred))

1.0


In [7]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 1000,l1_ratio=0)
lr.fit(x_train,train_y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1479: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty)


LogisticRegression(l1_ratio=0, max_iter=1000)

In [8]:
from sklearn.metrics import accuracy_score

y_pred = lr.predict(x_test)
print(accuracy_score(test_y,y_pred))

1.0
